In [110]:
import sys, os
import glob
from pprint import pprint
import csv
from pathlib import Path
import numpy as np
import pandas as pd

In [111]:
subtype = 'HGG'

In [112]:
DATA = Path(f'/data-nas/brains/MICCAI_BraTS2018/interim/{subtype}')

In [113]:
studies_path = sorted(glob.glob(str(DATA)+'/*'))
total_studies = len(studies_path)

In [114]:
from utils import write_nifti, load_nifti

In [115]:
def unique_values(studies_path):
 
    count_values = {}
    data_list = []
    for path in studies_path:
        study_id = (path.split("/")[-1])
        imgfn = (path + "/"+ study_id +"_seg.nii.gz")
        
        img, rnx, rny, rnz, data, affine = load_nifti(imgfn)
        
        #np_arr = np.unique(data) #unqiue values in a matrix 
        unique_arr, counts_arr = np.unique(data, return_counts=True)

        for i in unique_arr: 
            if i in count_values: count_values[i] += 1 
            else: count_values[i] = 1
        data_list.append([unique_arr,counts_arr])
                
    return count_values, data_list

In [116]:
dic_classes, data_list = unique_values(studies_path)
dic_classes

{0: 210, 1: 210, 2: 210, 4: 210}

In [117]:
info = []
for img_info in data_list: 
    total_size = img_info[1].sum()
    total = 0
    save_info = {}
    for idx,cls in enumerate(img_info[0]): 
        class_size = img_info[1][idx]
        size_in_img = class_size/total_size
        total += size_in_img
        save_info[cls] = size_in_img
    info.append(save_info)

In [100]:
df = pd.DataFrame(info)
df.head()

,0,1,2,4
0,0.994408,0.002383,0.001208,0.002
1,0.996590,0.000503,0.002907,NaN
2,0.992820,0.000619,0.006562,NaN
3,0.997884,0.001045,0.001071,NaN
4,0.993084,0.004862,0.002054,NaN


In [101]:
class_4 = df.isna().sum()[4]
df.isna().sum()

0     0
1     0
2     1
4    27
dtype: int64

In [105]:
df[2].fillna(0, inplace=True)

In [106]:
#df = df[np.isfinite(df[4])] #fjerne radene med nan verdier i klasse 4 
#total_studies = len(studies_path) - class_4

In [107]:
total_studies = len(studies_path)
df[4].fillna(0, inplace=True)
df.isna().sum()

0    0
1    0
2    0
4    0
dtype: int64

In [108]:
avg_tumor_size = {}
for column in df:
    avg_tumor_size[column] =  df[column].sum() / total_studies

In [109]:
avg_tumor_size #uten de 27 bildene med NaN

{0: 0.9875614471326164,
 1: 0.005933585722819593,
 2: 0.005974919354838709,
 4: 0.0005300477897252091}

In [93]:
avg_tumor_size #LGG  #uten de 27 bildene med NaN

{0: 0.9863543626792115,
 1: 0.0064376843451314225,
 2: 0.00637975330421147,
 4: 0.0008281996714456392}

# Check depth

In [118]:
def img_depth(studies_path):
 
    values = []
    for path in studies_path:
        study_id = (path.split("/")[-1])
        imgfn = (path + "/"+ study_id +"_seg.nii.gz")
        
        img, rnx, rny, rnz, data, affine = load_nifti(imgfn)
        depth = data.shape[2]
        values.append(depth)
    return values

In [119]:
depth_values = img_depth(studies_path)

In [121]:
np.unique(depth_values, return_counts=True)

(array([155]), array([210]))